In [ ]:
import os
import pandas as pd
import gym
from gym import spaces
import numpy as np
import pickle
import pprint
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


# Load Data

In [ ]:
#COnstants
last_minutes =  15

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
!pip install stable-baselines3[extra]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 9.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/trading/dataset.zip'

In [ ]:

from collections import defaultdict
def load_data(directory):
    data_by_day = {}
    indexToDayTime = defaultdict(dict)
    for day_dir in sorted(os.listdir(directory)):
        day_path = os.path.join(directory, day_dir)
        if os.path.isdir(day_path):
            csv_files = sorted(os.listdir(day_path))
            daily_data = []

            for csv_file in csv_files:
                csv_path = os.path.join(day_path, csv_file)
                df = pd.read_csv(csv_path)
                if len(df) < 7:
                    continue
                spot_price = df['Underlying LTP'].iloc[0]
                atm_strike = round(spot_price / 50) * 50
                df_filtered = df[(df['Strike Price'] >= atm_strike - 300) & (df['Strike Price'] <= atm_strike + 300)]

                # Check for NaN values in the relevant rows
                if df_filtered.isnull().values.any() or len(df_filtered)<13:
                    continue
                indexToDayTime[day_dir][len(daily_data)] = csv_file.replace('.csv', '')
                daily_data.append(df_filtered)

                # print(f"Loaded {csv_file} for day {day_dir}")

            if daily_data:
                data_by_day[day_dir] = daily_data

    return data_by_day, indexToDayTime

# Load data
data_directory = '/content/dataset'
data_by_day, indexToDayTime = load_data(data_directory)




In [ ]:
print(len(data_by_day))

218


In [ ]:
# save  data_by_day to a pickle file to MyDrive

import pickle

with open('/content/drive/MyDrive/trading/data_by_day.pkl', 'wb') as f:
    pickle.dump(data_by_day, f)
with open('/content/drive/MyDrive/trading/indexToDayTime.pkl', 'wb') as f:
    pickle.dump(indexToDayTime, f)

In [ ]:
## load data_by_day from pickle
import pickle
with open('/content/drive/MyDrive/trading/data_by_day.pkl', 'rb') as f:
    data_by_day = pickle.load(f)
with open('/content/drive/MyDrive/trading/indexToDayTime.pkl', 'rb') as f:
    indexToDayTime = pickle.load(f)

# Option ENV

In [ ]:
import gym
from gym import spaces
import numpy as np
import pandas as pd

action_descriptions = {
    0: "Hold",
    1: "Buy Call ATM",
    2: "Buy Call ATM+50",
    3: "Buy Call ATM+100",
    4: "Sell Call ATM",
    5: "Sell Call ATM+50",
    6: "Sell Call ATM+100",
    7: "Buy Put ATM",
    8: "Buy Put ATM-50",
    9: "Buy Put ATM-100",
    10: "Sell Put ATM",
    11: "Sell Put ATM-50",
    12: "Sell Put ATM-100",
    13: "Buy Call ATM-50",
    14: "Buy Call ATM-100",
    15: "Sell Call ATM-50",
    16: "Sell Call ATM-100",
    17: "Square Off"
}


class OptionTradingEnv(gym.Env):
    def __init__(self, data_by_day):
        super(OptionTradingEnv, self).__init__()
        self.data_by_day = data_by_day
        self.days = sorted(data_by_day.keys())
        self.current_day_index = 0
        self.current_step = 0
        self.current_day_data = None
        self.day_key = self.days[self.current_day_index]
        self.realised_profit = 0
        # Action space: 18 discrete actions (including the new square off action)
        self.action_space = spaces.Discrete(18)

        # Observation space: 10 state variables
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(10,), dtype=np.float32)

        self.positions = {'call': {}, 'put': {}}  # Track positions
        self.current_profit = 0
        self.reward = 0
        self.maximum_profit = 0
        self.nTrades = 0
        self.reset()
        self.previous_profit = 0
        self.constant_reward = 80

    def reset(self):
        if self.current_day_index >= len(self.days):
            self.current_day_index = 0
        self.day_key = self.days[self.current_day_index]
        self.current_day_data = self.data_by_day[self.day_key]
        self.current_step = 0
        self.realised_profit=0
        self.current_day_index += 1
        print(f'Current Positions {self.positions}')
        self.positions = {'call': {}, 'put': {}}  # Reset positions
        print(f"trades {self.nTrades} Current profit: {self.current_profit}, Maximum profit: {self.maximum_profit}")

        self.current_profit = 0
        self.maximum_profit = 0
        self.state = self._get_state(self.current_step)
        self.nTrades = 0
        return self.state

    def step(self, action):
        valid_action = self._take_action(action)
        if not valid_action:
            reward = -10000  # Negative reward for invalid action
            return self.state, reward, False, {}

        self.current_step += 1
        if self.current_step + last_minutes >= len(self.current_day_data):
            done = True
            reward = self.current_profit
            # self.animate()  # Call animate at the end of the episode
            return self.state, reward, done, {}

        done = False
        reward = self.reward # self._calculate_reward(action)
        self.state = self._get_state(self.current_step)
        # self.animate(action)  # Call animate after each step
        return self.state, reward, done, {}



    def _take_action(self, action):
        if action == 17:
            # Square off all positions
            self.current_profit = self._calculate_current_profit() + self.realised_profit
            self.positions = {'call': {}, 'put': {}}
            self.realised_profit += self._calculate_current_profit()
            self.reward = self.realised_profit
            return True

        current_prices = self.current_day_data[self.current_step]
        spot_price = current_prices['Underlying LTP'].values[0]
        atm_strike = round(spot_price / 50) * 50

        if action == 0:
            # Simply update current profit without making any new positions
            self.previous_profit = self.current_profit
            self.current_profit = self._calculate_current_profit() + self.realised_profit
            self.maximum_profit = max(self.maximum_profit, self.current_profit)
            self.reward = self.current_profit - self.previous_profit
            return True

        strike_offset = (action-1) % 4
        is_call = action < 9
        is_buy = (action % 9) <= 4
        self.constant_reward -= self.constant_reward * 0.9

        if is_call:
            strike_price = atm_strike + (strike_offset * 50)
            option_type = 'call'
        else:
            strike_price = atm_strike - (strike_offset * 50)
            option_type = 'put'

        option_key = strike_price
        ltp_column = f'{"CALL" if is_call else "PUT"}_LTP'

        current_price = current_prices.loc[current_prices['Strike Price'] == strike_price, ltp_column]
        if current_price.empty:
            return False  # Invalid action due to missing strike price

        current_price = current_price.values[0]

        # Check position limit
        if option_key in self.positions[option_type]:
            if is_buy and self.positions[option_type][option_key]['quantity'] >= 9:
                return False
            if not is_buy and self.positions[option_type][option_key]['quantity'] <= -9:
                return False

        if option_key in self.positions[option_type]:
            self.positions[option_type][option_key]['quantity'] += 1 if is_buy else -1
            self.positions[option_type][option_key]['prices'].append(-current_price if is_buy else current_price)
        else:
            self.positions[option_type][option_key] = {
                'quantity': 1 if is_buy else -1,
                'prices': [-current_price if is_buy else current_price]
            }
        self.nTrades += 1
        # Update current and maximum profit
        self.previous_profit = self.current_profit
        self.current_profit = self._calculate_current_profit() + self.realised_profit
        self.maximum_profit = max(self.maximum_profit, self.current_profit)
        self.reward = -current_price if is_buy else current_price
        return True

    def _get_state(self, step):
        row = self.current_day_data[step]

        spot_price = row['Underlying LTP'].values[0]

        atm_strike = round(spot_price / 50) * 50

        gamma_call = row[row['Strike Price'] == atm_strike]['GAMMA_CALL'].values[0]
        gamma_put = row[row['Strike Price'] == atm_strike]['GAMMA_PUT'].values[0]
        # print(row[row['Strike Price'] == atm_strike])
        del_vega = row['Vega_Call'].sum() - row['Vega_Put'].sum()
        del_delta = row['DELTA_CALL'].sum() - row['DELTA_PUT'].sum()

        call_buy_position = self._encode_positions(self.positions['call'], atm_strike, 'buy', 'call')
        put_buy_position = self._encode_positions(self.positions['put'], atm_strike, 'buy', 'put')
        call_sell_position = self._encode_positions(self.positions['call'], atm_strike, 'sell', 'call')
        put_sell_position = self._encode_positions(self.positions['put'], atm_strike, 'sell', 'put')

        return np.array([gamma_call, gamma_put, del_vega, del_delta, self.current_profit, self.maximum_profit, call_buy_position, put_buy_position, call_sell_position, put_sell_position], dtype=np.float32)

    def _calculate_current_profit(self):
        current_prices = self.current_day_data[self.current_step]
        profit = 0
        for option_type in ['call', 'put']:
            for strike_price, position in self.positions[option_type].items():
                ltp_column = f'CALL_LTP' if option_type == 'call' else 'PUT_LTP'
                current_price = current_prices.loc[current_prices['Strike Price'] == strike_price, ltp_column]
                if current_price.empty:
                    continue

                current_price = current_price.values[0]
                net_quantity = position['quantity']
                if net_quantity > 0 or net_quantity < 0:
                    # Net buy position
                    avg_buy_price = sum(position['prices']) / net_quantity
                    profit += net_quantity * (current_price + avg_buy_price)
                if  net_quantity < 0:
                    # Net sell position
                    avg_sell_price = sum(position['prices']) / net_quantity
                    profit += net_quantity * (current_price - avg_sell_price)
                elif net_quantity == 0:
                    # hold
                    profit += sum(position['prices'])

        return profit
    def _encode_positions(self, positions, atm_strike, pos_type, strike_type):
        pos_encoding = 0
        for i in range(4):
            strike_price = atm_strike + (i * 50) if strike_type == 'call' else atm_strike - (i * 50)
            if strike_price in positions and pos_type == 'buy' and positions[strike_price]['quantity'] > 0:
                pos_encoding += positions[strike_price]['quantity'] *-1 * (10 ** (3 - i))
            if strike_price in positions and pos_type == 'sell' and positions[strike_price]['quantity'] < 0 :
                pos_encoding += positions[strike_price]['quantity'] * (10 ** (3 - i))
        return pos_encoding

    def _calculate_reward(self, action):
        interim_reward = self.current_profit - self.maximum_profit + self.constant_reward
        if interim_reward < -40:
            interim_reward *= 3
        elif interim_reward < -20:
            interim_reward *= 2
        elif interim_reward < 0:
            interim_reward *= 1.5
        # if action_descriptions[action].contains("Buy"):
        #     interim_reward
        # else:
        #     interim_reward = 0

        # Reward for taking the square off action when the overall profit is less than -100
        if action == 17 and self.current_profit < -100:
            interim_reward += 50  # Arbitrary positive reward for triggering stop loss

        return interim_reward

    def animate(self, action=None):
      if self.current_step >= len(self.current_day_data):
          return

      # Get current day data
      current_prices = self.current_day_data[self.current_step]
      spot_price = current_prices['Underlying LTP'].values[0]
      date_time = self.day_key + ' ' + indexToDayTime[self.day_key][self.current_step]


      strike_prices = current_prices['Strike Price'].tolist()
      call_prices = current_prices['CALL_LTP'].tolist()
      put_prices = current_prices['PUT_LTP'].tolist()

      call_buy_positions = []
      call_sell_positions = []
      put_buy_positions = []
      put_sell_positions = []

      for strike in strike_prices:
          call_key = strike #(strike, 'call')
          put_key = strike #(strike, 'put')

          call_buy_positions.append(self.positions['call'].get(call_key, {'quantity': 0})['quantity'])
          call_sell_positions.append(-self.positions['call'].get(call_key, {'quantity': 0})['quantity'])
          put_buy_positions.append(self.positions['put'].get(put_key, {'quantity': 0})['quantity'])
          put_sell_positions.append(-self.positions['put'].get(put_key, {'quantity': 0})['quantity'])

      data = {
          'Call Buy': call_buy_positions,
          'Call Sell': call_sell_positions,
          'Call LTP': call_prices,
          'Strike Price': strike_prices,
          'Put LTP': put_prices,
          'Put Sell': put_sell_positions,
          'Put Buy': put_buy_positions
      }

      df = pd.DataFrame(data)
      df.loc[len(df)] = ['', '', '', '', '', 'Current Profit', int(self.current_profit)]
      df.loc[len(df)] = ['', '', '', '', '', 'Max Profit',int(self.maximum_profit)]
      df.loc[len(df)] = ['', '', '', '', '', 'Spot Price', spot_price]
      df.loc[len(df)] = ['', '', '', '', '', 'Date Time', date_time]
      if action is not None:
          df.loc[len(df)] = ['', '', '', '', '', 'Action', action_descriptions[action]]

      clear_output(wait=True)
      fig, ax = plt.subplots()
      ax.axis('off')
      table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')
      table.auto_set_font_size(False)
      table.set_fontsize(10)
      table.scale(1.2, 1.2)

      display(plt.gcf())
      plt.close(fig)
      del fig
      del table
      del ax
      del df



env = OptionTradingEnv(data_by_day)


Current Positions {'call': {}, 'put': {}}
trades 0 Current profit: 0, Maximum profit: 0


In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Vectorized environment
vec_env = DummyVecEnv([lambda: env])

# Initialize the PPO agent
model = PPO('MlpPolicy', vec_env, verbose=1)

# Train the agent
model.learn(total_timesteps=100000)

# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, vec_env, n_eval_episodes=10)
print(f'Mean reward: {mean_reward} +/- {std_reward}')

# Save the model
model.save('ppo_option_trading')

# Load the model (if needed)
# model = PPO.load('ppo_option_trading')


Using cuda device
Current Positions {'call': {}, 'put': {}}
trades 0 Current profit: 0, Maximum profit: 0
Current Positions {'call': {18300: {'quantity': -2, 'prices': [-46.5, 46.2, 46.2, 44.95]}, 18350: {'quantity': 2, 'prices': [-29.8, -29.8]}, 18200: {'quantity': 2, 'prices': [-92.0, -92.5]}, 18250: {'quantity': 0, 'prices': [-65.95, 63.8]}}, 'put': {18200: {'quantity': 1, 'prices': [-64.2]}}}
trades 255 Current profit: -186.95000000000002, Maximum profit: 1.9000000000000057
Current Positions {'call': {18200: {'quantity': -1, 'prices': [14.9]}}, 'put': {}}
trades 223 Current profit: -29.8, Maximum profit: 73.99999999999997
Current Positions {'call': {17900: {'quantity': -1, 'prices': [105.65]}}, 'put': {17700: {'quantity': 1, 'prices': [-37.55]}}}
trades 289 Current profit: -215.0, Maximum profit: 0
Current Positions {'call': {18050: {'quantity': 0, 'prices': [-33.35, 33.15]}, 18000: {'quantity': -1, 'prices': [49.75]}, 17900: {'quantity': 2, 'prices': [-93.05, -93.35]}}, 'put': {17

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Current Positions {'call': {}, 'put': {}}
trades 114 Current profit: -725.95, Maximum profit: 0


KeyboardInterrupt: 

In [ ]:
for day in data_by_day:


    ls = data_by_day[day]
    for row in ls:

      spot_price = row['Underlying LTP'].values[0]

      atm_strike = round(spot_price / 50) * 50
      try :
          gamma_call = row[row['Strike Price'] == atm_strike]['GAMMA_CALL'].values[0]
      except:
          # print(day)
          print(row)
          break


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


   Strike Price  CALL_LTP  PUT_LTP  GAMMA_CALL  GAMMA_PUT   IV_CALL    IV_PUT  \
4         16850     333.0    27.65    0.000780   0.000781  0.253146  0.249218   
5         16900     288.1    35.90    0.000912   0.000906  0.240408  0.245820   
6         16950     247.9    45.75    0.001045   0.001034  0.236398  0.241387   
7         17000     210.0    58.20    0.001173   0.001154  0.232354  0.237722   
8         17050     175.0    72.85    0.001283   0.001263  0.228802  0.233094   

   VOLUME_CALL  VOLUME_PUT  DELTA_CALL  ...  THETA_CALL  THETA_PUT  RHO_CALL  \
4          550      172050    0.832486  ...  -22.202315 -19.110513         0   
5        13200      713350    0.801696  ...  -23.244831 -21.531667         0   
6         5850      236800    0.755841  ...  -25.426028 -23.619744         0   
7        97450     1232950    0.703164  ...  -27.273159 -25.480951         0   
8        45950      446300    0.643624  ...  -28.682410 -26.707618         0   

   RHO_PUT  Underlying LTP  CALL